In [51]:
from distributed import Client
import dask.dataframe as dd

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51144,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:51163,Total threads: 2
Dashboard: http://127.0.0.1:51164/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:51151,


## 1. Import Mongo Data

In [3]:
from dask_mongo import read_mongo
import urllib

In [4]:
# Replace the username, password, and cluster address with your own connection details
host_uri = "mongodb+srv://richard:" + urllib.parse.quote("Rp@976559MO") + "@cluster0.ffttf.mongodb.net/myFirstDatabase?retryWrites=true&w=majority"

In [5]:
bag = read_mongo(
    connection_kwargs={"host": host_uri},
    database="sample_airbnb",
    collection="listingsAndReviews",
    chunksize=500,
)

In [6]:
bag.take(1)

({'_id': '10006546',
  'listing_url': 'https://www.airbnb.com/rooms/10006546',
  'name': 'Ribeira Charming Duplex',
  'summary': 'Fantastic duplex apartment with three bedrooms, located in the historic area of Porto, Ribeira (Cube) - UNESCO World Heritage Site. Centenary building fully rehabilitated, without losing their original character.',
  'space': 'Privileged views of the Douro River and Ribeira square, our apartment offers the perfect conditions to discover the history and the charm of Porto. Apartment comfortable, charming, romantic and cozy in the heart of Ribeira. Within walking distance of all the most emblematic places of the city of Porto. The apartment is fully equipped to host 8 people, with cooker, oven, washing machine, dishwasher, microwave, coffee machine (Nespresso) and kettle. The apartment is located in a very typical area of the city that allows to cross with the most picturesque population of the city, welcoming, genuine and happy people that fills the streets w

In [7]:
def process(record):
    try:
        yield {
            "description": record["description"],
            "review_rating": int(str(record["review_scores"]["review_scores_rating"])),
            #"accomodates": record["accommodates"],
            #"bedrooms": record["bedrooms"],
            #"price": float(str(record["price"])),
            #"country": record["address"]["country"],
        }
    except KeyError:
        pass

In [8]:
# Filter only apartments
b_flattened = (
    bag.filter(lambda record: record["property_type"] == "Apartment")
    .map(process)
    .flatten()
)

In [9]:
b_flattened.take(1)

({'description': 'Here exists a very cozy room for rent in a shared 4-bedroom apartment. It is located one block off of the JMZ at Myrtle Broadway.  The neighborhood is diverse and appeals to a variety of people.',
  'review_rating': 100},)

In [34]:
df3 = b_flattened.to_dataframe()

In [36]:
df3.head()

,description,review_rating
0,Here exists a very cozy room for rent in a sha...,100
1,"Murphy bed, optional second bedroom available....",94
2,"The Apartment has a living room, toilet, bedro...",98
3,Loft Suite Deluxe @ Henry Norman Hotel Located...,88
4,"Clean, fully furnish, Spacious 1 bedroom flat ...",100


## 2. Tokenization

In [10]:
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
from functools import partial

In [11]:
tokenizer = RegexpTokenizer(r'\w+')

In [12]:
# define processing functions
def extract_description(element):
    return element['description'].lower()

def extract_rating(element):
    return element['review_rating']

def filter_stopword(word, stopwords):
    return word not in stopwords

def filter_stopwords(tokens, stopwords):
    return list(filter(partial(filter_stopword, stopwords=stopwords), tokens))

In [13]:
# define set of stopwords
stopword_set = set(stopwords.words('english'))

In [15]:
# get cleaned, tokenized description texts
description_text = b_flattened.map(extract_description)

In [38]:
description_text.take(1)

('here exists a very cozy room for rent in a shared 4-bedroom apartment. it is located one block off of the jmz at myrtle broadway.  the neighborhood is diverse and appeals to a variety of people.',)

In [41]:
df1 = description_text.to_dataframe()

In [43]:
df1.head()

,0
0,here exists a very cozy room for rent in a sha...
1,"murphy bed, optional second bedroom available...."
2,"the apartment has a living room, toilet, bedro..."
3,loft suite deluxe @ henry norman hotel located...
4,"clean, fully furnish, spacious 1 bedroom flat ..."


In [39]:
description_text_tokens = description_text.map(tokenizer.tokenize)

In [40]:
description_text_tokens.take(1)

(['here',
  'exists',
  'a',
  'very',
  'cozy',
  'room',
  'for',
  'rent',
  'in',
  'a',
  'shared',
  '4',
  'bedroom',
  'apartment',
  'it',
  'is',
  'located',
  'one',
  'block',
  'off',
  'of',
  'the',
  'jmz',
  'at',
  'myrtle',
  'broadway',
  'the',
  'neighborhood',
  'is',
  'diverse',
  'and',
  'appeals',
  'to',
  'a',
  'variety',
  'of',
  'people'],)

In [44]:
df2 = description_text_tokens.to_dataframe()

In [ ]:
# something's going wrong with the columns here
description_text_tokens = description_text.map(tokenizer.tokenize)
descript_clean = description_text_tokens.map(partial(filter_stopwords, stopwords=stopword_set))

In [16]:
descript_clean.take(1)

(['exists',
  'cozy',
  'room',
  'rent',
  'shared',
  '4',
  'bedroom',
  'apartment',
  'located',
  'one',
  'block',
  'jmz',
  'myrtle',
  'broadway',
  'neighborhood',
  'diverse',
  'appeals',
  'variety',
  'people'],)

In [37]:
df4 = descript_clean.to_dataframe()
df4.head()

distributed.worker - WARNING - Compute Failed
Function:  to_dataframe
args:      ([['exists', 'cozy', 'room', 'rent', 'shared', '4', 'bedroom', 'apartment', 'located', 'one', 'block', 'jmz', 'myrtle', 'broadway', 'neighborhood', 'diverse', 'appeals', 'variety', 'people'], ['murphy', 'bed', 'optional', 'second', 'bedroom', 'available', 'wifi', 'available', 'hulu', 'netflix', 'tv', 'eat', 'kitchen', 'bathroom', 'great', 'shower', 'bath', 'washer', 'dryer', 'basement', 'new', 'york', 'city', 'great', 'neighborhood', 'many', 'terrific', 'restaurants', 'bakeries', 'bagelries', 'within', 'easy', 'walking', 'distance', 'restaurants', 'cuisines', 'india', 'thailand', 'japan', 'china', 'mexico', 'south', 'america', 'europe', 'well', 'many', 'small', 'independent', 'stores', 'line', 'broadway', 'chain', 'stores', 'urban', 'outfitters', 'clothing', 'whole', 'foods', 'groceries', 'sephora', 'cosmetics', 'michaels', 'crafts', 'modell', 'sporting', 'goods', 'equidistant', 'central', 'park', 'riversi

ValueError: 19 columns passed, passed data had 167 columns

## 3. Lemmatization

In [17]:
import spacy

In [18]:
nlp = spacy.load("en_core_web_sm")

In [19]:
def lemmatize(text, nlp=nlp):
    doc = nlp(" ".join(text))
    lemmatized = [token.lemma_ for token in doc]
    return lemmatized

In [20]:
lemmas = descript_clean.map(lemmatize)

In [21]:
lemmas.take(1)

(['exist',
  'cozy',
  'room',
  'rent',
  'share',
  '4',
  'bedroom',
  'apartment',
  'locate',
  'one',
  'block',
  'jmz',
  'myrtle',
  'broadway',
  'neighborhood',
  'diverse',
  'appeal',
  'variety',
  'people'],)

## 4. Topic Modelling

To perform topic modelling, we will have to:
1. Create an array containing only the lemmatized text
2. Create a Bag of Words dictionary (filtering for extreme cases)
3. Map our documents to the BOW dictionary

OR
1. use the HashingVectorizer to turn text into a matrix of token occurrences

Then we'll use sklearn instead of Gensim with the n_jobs parameter and the Dask backend (connected to a Coiled cluster).



1. Input Bag to LDA? or Array?
2. Use Sklearn instead of Gensim?
3. njobs with dask backend?

## 4.1. Use Lemmas > Array > BOW

### Turn Bag into Array

In [27]:
type(lemmas)

dask.bag.core.Bag

In [46]:
lemmas_df = lemmas.to_dataframe(meta={'lemmas': 'object'})

In [48]:
lemmas_df.head()

distributed.worker - WARNING - Compute Failed
Function:  to_dataframe
args:      ([['exist', 'cozy', 'room', 'rent', 'share', '4', 'bedroom', 'apartment', 'locate', 'one', 'block', 'jmz', 'myrtle', 'broadway', 'neighborhood', 'diverse', 'appeal', 'variety', 'people'], ['murphy', 'bed', 'optional', 'second', 'bedroom', 'available', 'wifi', 'available', 'hulu', 'netflix', 'tv', 'eat', 'kitchen', 'bathroom', 'great', 'shower', 'bath', 'washer', 'dryer', 'basement', 'new', 'york', 'city', 'great', 'neighborhood', 'many', 'terrific', 'restaurant', 'bakerie', 'bagelrie', 'within', 'easy', 'walking', 'distance', 'restaurant', 'cuisine', 'india', 'thailand', 'japan', 'china', 'mexico', 'south', 'america', 'europe', 'well', 'many', 'small', 'independent', 'store', 'line', 'broadway', 'chain', 'store', 'urban', 'outfitter', 'clothe', 'whole', 'food', 'grocery', 'sephora', 'cosmetic', 'michael', 'craft', 'modell', 'sporting', 'good', 'equidistant', 'central', 'park', 'riverside', 'park', 'walk', 

ValueError: 1 columns passed, passed data had 167 columns

Casting this to a dataframe doesn't work because each element of the list of lemmas is getting cast to its own df column. We need the whole list of lemmas to end up in a single column.

In [ ]:
lemmas_array = lemmas_df.to_numpy()

### Create BOW Dictionary

### Create tf-idf mapping

## 4.2. HashingVectorizer

In [49]:
from dask_ml.feature_extraction.text import HashingVectorizer

In [52]:
ddf = dd.read_parquet(
    's3://coiled-datasets/airbnb-monogo/description-and-ratings.parquet',
    engine="pyarrow",
)

In [53]:
ddf.head()

,description,review_rating
0,Here exists a very cozy room for rent in a sha...,100
1,"Murphy bed, optional second bedroom available....",94
2,"The Apartment has a living room, toilet, bedro...",98
3,Loft Suite Deluxe @ Henry Norman Hotel Located...,88
4,"Clean, fully furnish, Spacious 1 bedroom flat ...",100


In [54]:
X = ddf['description'].to_dask_array(lengths=True)
y = ddf['review_rating'].to_dask_array(lengths=True)

In [55]:
vect = HashingVectorizer()

In [56]:
X_vect = vect.fit_transform(X)

In [59]:
X_vect.compute_chunk_sizes()

dask.array<_transformer, shape=(2681, 1048576), dtype=float64, chunksize=(264, 1048576), chunktype=scipy.csr_matrix>

In [61]:
X_vect.blocks[0].compute()

<221x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 19632 stored elements in Compressed Sparse Row format>

Each block in X is a scipy.sparse matrix.

## 4.3. CountVectorizer

In [65]:
from dask_ml.feature_extraction.text import CountVectorizer
import dask.bag as db

"The Dask-ML implementation currently requires that raw_documents is a dask.bag.Bag of documents (lists of strings)."

In [67]:
description_text.take(1)

('here exists a very cozy room for rent in a shared 4-bedroom apartment. it is located one block off of the jmz at myrtle broadway.  the neighborhood is diverse and appeals to a variety of people.',)

`description_text` is a Dask Bag of documents

In [68]:
vectorizer = CountVectorizer()

In [70]:
X = vectorizer.fit_transform(description_text)

In [76]:
X_local = X.compute().toarray()

Just like `HashingVectorizer`, the `CountVectorizer` outputs a Dask Array where each chunk is a scipy.sparse.matrix

## 5. LDA

## 5.1. Sklearn

In [77]:
from sklearn.decomposition import LatentDirichletAllocation

In [78]:
lda = LatentDirichletAllocation(
    n_components=5,
    random_state=42,
)

In [79]:
import joblib

In [81]:
%%time
with joblib.parallel_backend("dask", scatter=[X]):
    lda.fit(X_local)

CPU times: user 1.47 s, sys: 457 ms, total: 1.93 s
Wall time: 4.91 s
